In [12]:
from ollama import chat,list
from ollama import ChatResponse
from ollama import Client
import asyncio
from ollama import AsyncClient
import requests
from bs4 import BeautifulSoup
from ollama import chat
from pydantic import BaseModel 
from typing import Literal, Optional,List
def llama32():
  response: ChatResponse = chat(model='llama3.2', messages=[
    {
      'role': 'user',
      'content': 'Why is the sky blue? 用简体中文回答',
    },
  ])
  # print(response['message']['content'])
  # or access fields directly from the response object
  print(response.message.content)

def streaming():
  stream = chat(
      model='llama3.2',
      messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
      stream=True,
  )

  for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)
    
def client(content,model='llama3.2'):
  client = Client(
    host='http://localhost:11434',
    verify=False,  # 关闭 SSL 验证
    headers={'x-some-header': 'some-value'}
  )
  response = client.chat(model, messages=[
    {
      'role': 'user',
      'content': content if content else 'Why is the sky blue? 用简体中文回答',
    },
  ])
  print(response.message.content)
  
def asyncClient():
  async def chat():
    message = {'role': 'user', 'content': 'Why is the sky blue?'}
    response = await AsyncClient().chat(model='llama3.2', messages=[message])

  asyncio.run(chat())
  
  
def streamingAsync():
  async def chat():
    message = {'role': 'user', 
               'content': 'Why is the sky blue?'}
    async for part in await AsyncClient().chat(model='llama3.2', messages=[message], stream=True):
      print(part['message']['content'], end='', flush=True)

  asyncio.run(chat())
  


class Object(BaseModel):
    name: str
    confidence: float
    attributes: str

class ImageDescription(BaseModel):
    summary: str
    objects: List[Object]
    scene: str
    colors: List[str]
    time_of_day: Literal['Morning', 'Afternoon', 'Evening', 'Night']
    setting: Literal['Indoor', 'Outdoor', 'Unknown']
    text_content: Optional[str] = None
    
def image_roc():
    # 假设这是一个网页的 URL
    url = 'https://example.com/page-with-image'

    # 发送请求获取网页内容
    response = requests.get(url)
    html_content = response.text

    # 使用 BeautifulSoup 解析 HTML 内容
    soup = BeautifulSoup(html_content, 'html.parser')

    # 假设图片在某个特定的标签中，例如 <img> 标签
    image_tag = soup.find('img')  # 你可以使用更具体的条件来定位图片标签
    if image_tag and 'src' in image_tag.attrs:
        image_url = image_tag['src']
    else:
        raise ValueError("Image not found or no 'src' attribute in the image tag")

    # 如果图片 URL 是相对路径，转换为绝对路径
    if image_url.startswith('/'):
        base_url = '{uri.scheme}://{uri.netloc}'.format(uri=response.url)
        image_url = base_url + image_url

    # 将图片 URL 传递给 chat 函数
    response = chat(
        model='llama3.2-vision',
        format=ImageDescription.model_json_schema(),  # 传递响应的 schema
        messages=[
            {
                'role': 'user',
                'content': 'Analyze this image and describe what you see, including any objects, the scene, colors and any text you can detect.',
                'images': [image_url],
            },
        ],
        options={'temperature': 0},  # 设置温度为 0 以获得更确定的输出
    )

    # 解析响应内容
    image_description = ImageDescription.model_validate_json(response.message.content)
    print(image_description)

In [28]:
for m in list():
    (a,b)=m
    for k in b:
        print(f'{k.model}')

llama3.3:latest
opencoder:latest
marco-o1:latest
llama3.2:latest
llama3.2-vision:latest
qwen2.5-coder:latest
mistral:latest
qwq:latest


In [13]:
s='''我国甲醇进口量近期呈现较为明显的下降态势。期货日报记者了解到，10月和11月初，我国甲醇月均进口量为130万吨左右，而11月底，甲醇进口量明显下降，11月伊朗装船量预计下降至70万吨左右，12月全国进口量预计在105万吨左右，相对前两个月下降明显。采访中，期货日报记者了解到，我国甲醇进口量下降的主要原因是伊朗装置停车。“目前伊朗装置处于轮流停车状态，开工率较低，今年限气相对去年来说较早。”永安期货分析师周禹通表示，伊朗今年限气时间提前是因为燃料短缺、电力紧张。从供需情况看，伊朗发货量降低的另一个原因是国内港口甲醇库存较高，甲醇进口利润较低，伊朗装置开工意愿下降。此外，10月以来人民币兑美元汇率不断贬值，甲醇进口成本同步走高。“这导致甲醇进口利润较差，国际贸易商更愿意将紧张的货源运往高价区域套利，运至中国货源大幅减少。”国信期货分析师郑淅予如是说。与往年相比，今年甲醇市场整体表现出进口量下降、进口利润减少、港口库存高的特点。数据显示，1—10月，我国甲醇累计进口量为1129.02万吨，其中，10月份进口量为122.72万吨，环比下降1.73%，同比下降6.02%。此外，上半年进口利润为56元/吨，下半年大部分时间进口利润出现倒挂，年内平均进口利润为-12元/吨。“目前国内甲醇进口市场呈现出一定的波动性和不确定性，受到天气、地缘政治、装置运行状况等多重因素影响，进口量难以保持平稳。”华融融达期货分析师彭杰斌表示，一方面，国际甲醇装置检修频繁，导致产量偏低，影响了进口量；另一方面，伊朗冬季限气预期导致甲醇产量减少，影响了供应量；此外，国内甲醇下游需求表现疲软，尤其是传统消费领域表现乏力，导致进口需求减少。同时，今年全球甲醇的货源流向也发生了变化。据郑淅予介绍，2024年全球甲醇产能持续扩张，新增产能主要来自中国、美国和伊朗等国家，各区域供需结构发生变化，全球整体货源流向调整后再次达到平衡。今年欧洲自身供应缩减情况愈发明显，欧洲甲醇价格维持高位，国际甲醇厂商倾向于去欧洲套利，对中国的供应意愿下降。在彭杰斌看来，今年甲醇市场进口缩量的原因更为复杂，甲醇进口抵港量同比下降可能是地缘冲突、海外装置运行状况及国内需求变化等多重因素共同作用的结果。由于进口持续缩量，今年中国港口甲醇库存较往年大幅下降，年均值仅有85万吨，较2023年库存水平下降7.3%。“进口甲醇缩量首先冲击港口市场，当前港口总库存为120.05万吨，同比增加23.38万吨，进口缩量有助于港口去库。此外，进口缩量也会助推价格上行，进而影响国内甲醇企业的生产决策，如增加国内甲醇的生产量以弥补供应缺口，这将表现为港口与内地甲醇套利空间缩小。”彭杰斌称。周禹通认为，港口库存较高压制了甲醇的基差和估值，若后期港口库存降至低位，甲醇估值将有所抬升。在此背景下，甲醇价格将呈现震荡偏强的走势。同样，在郑淅予看来，当前内地代表企业的甲醇库存同样不高，且新增宝丰烯烃装置仍有外采需求，内地市场价格重心走高，对港口价格同样有一定支撑。事实上，伊朗提前限气导致甲醇去库，盘面已经有所反应。“甲醇价格上涨较多，MTO利润压缩较明显，但现实库存仍处于高位，尚未开始去库，投资者可以观察后期库存情况。”周禹通表示，若后期甲醇价格仍处于高位，下游MTO利润较低但库存去化缓慢，则价格可能存在回调压力。展望后市，彭杰斌认为，投资者一方面需注意供应过剩的风险，国内甲醇供应压力持续增加，可能导致价格承压；另一方面，注意需求疲软的风险，下游需求改善有限，尤其是传统消费领域表现乏力，可能削弱价格支撑。“目前港口库存维持高位，若后市去库力度不及预期，可能增加甲醇价格下行的压力。”他说。“短期看来，甲醇进口量将下降，但长期看来，随着非伊装置检修回归，限气季节过去，进口量反而存在回升的可能。”周禹通称。“从估值角度来看，煤价冬季存在一定支撑，国际甲醇供应量下降，甲醇价格重心或将上移。”郑淅予称。*END*责任编辑：孙亚宁 主管：李红珠推荐阅读：凌晨爆发，中国资产飙升！“立场重大转变”，特朗普最新宣布！伦锌库存骤降近40%！国际油价怎么走？中办、国办最新印发！央行等七部门联合发文：全部纳入监管→   '''
                                                         

In [35]:
client(f'生成文章概要：{s}')

本文主要讨论了中国甲醇进口量的下降趋势，原因包括伊朗装置停车、国内港口甲醇库存较高和人民币兑美元汇率贬值。随着伊朗装置停车，中国甲醇进口量明显下降，预计11月伊朗装船量将减少至70万吨左右，12月全国进口量可能会进一步下降。分析师认为，国内港口甲醇库存较高使得甲醇进口利润较低，国际贸易商更愿意将紧张的货源运往高价区域套利，导致进口缩量。


In [31]:
client(content= f'生成文章概要：{s}',model='llama3.3')

文章概要：

我国甲醇进口量近期呈现明显下降态势，10月和11月初的月均进口量为130万吨左右，但11月底和12月全国进口量预计分别下降至70万吨和105万吨左右。主要原因包括伊朗装置停车、国内港口甲醇库存较高、进口利润较低，以及人民币兑美元汇率贬值导致的进口成本增加。

今年甲醇市场整体表现出进口量下降、进口利润减少、港口库存高的特点。2024年全球甲醇产能持续扩张，新增产能主要来自中国、美国和伊朗等国家，但欧洲自身供应缩减情况愈发明显，对中国的供应意愿下降。

由于进口持续缩量，今年中国港口甲醇库存较往年大幅下降。进口甲醇缩量首先冲击港口市场，有助于港口去库，也会助推价格上行，影响国内甲醇企业的生产决策。

未来，投资者需要注意供应过剩和需求疲软的风险，同时关注后期库存情况、估值变化以及全球供需结构调整对甲醇价格的影响。


In [34]:
client(content= f'生成文章概要：{s}',model='qwq')

最近我注意到国内甲醇的进口量有明显的下降趋势。作为一个期货市场的参与者，我对这个现象感到好奇，想要深入了解一下背后的原因以及它对市场可能产生的影响。

首先，我了解到10月和11月初，我国甲醇的月均进口量大约是130万吨左右，但到了11月底，进口量明显减少，预计12月份的进口量会进一步下降到105万吨左右。这说明进口量在短时间内有了较大幅度的下滑。

我开始思考，是什么原因导致了这种下降呢？通过查阅一些资料和咨询行业内的分析师，我发现主要原因在于伊朗甲醇装置的停车。伊朗是中國甲醇的重要供应国，他们的装置如果停产或者减产，自然会影响到对中国市场的供货。

永安期货的分析师周禹通提到，伊朗的装置目前处于轮流停车的状态，开工率较低，而且今年的限气时间比去年要早。他说这是由于燃料短缺和电力紧张导致的。我觉得这个解释很有道理，因为能源紧缺确实会影响生产活动。

另外，从供需的角度来看，伊朗国内港口的甲醇库存较高，而进口利润较低，这也可能导致伊朗的装置开工意愿下降。也就是说，如果出口没有很好的利润空间，他们可能更愿意减少生产或者将产品留在国内市场。

再者，10月以来人民币兑美元的汇率不断贬值，这导致甲醇的进口成本上升。国信期货的分析师郑淅予指出，进口利润变差，国际贸易商可能会选择将有限的货源运往价格更高的地区套利，从而减少了对中国市场的供应。

综合这些因素，我可以看出，今年甲醇市场整体呈现出进口量下降、进口利润减少以及港口库存较高的特点。与往年相比，这种趋势是比较明显的。

根据海关的数据，1到10月份，我国甲醇累计进口量为1129.02万吨，其中10月份进口了122.72万吨，环比下降1.73%，同比下降6.02%。此外，上半年的进口利润是56元/吨，而下半年大部分时间进口利润出现倒挂，全年的平均进口利润是-12元/吨。

这说明进口甲醇在国内市场的竞争力在减弱，因为成本更高，利润更低，自然会减少进口量。

华融融达期货的分析师彭杰斌也提到，国内甲醇进口市场呈现出一定的波动性和不确定性，受到天气、地缘政治、装置运行状况等因素的影响，进口量难以保持平稳。

他列举了几个原因：一是国际甲醇装置检修频繁，产量偏低，影响了进口量；二是伊朗冬季限气预期导致甲醇产量减少，供应下降；三是国内甲醇下游需求表现疲软，尤其是传统消费领域不景气，减少了进口需求。

同时，今年全球甲醇的货源流向

In [33]:
client(content= f'生成文章概要：{s}',model='mistral')

在过去几个月，我国甲醇进口量显示出明显的下降趋势。根据期货日报记者的了解，该情况可能是由于伊朗提前限气、中东地区的冲突以及港口市场上市商品的供应过剩等因素所致。

根据数据显示，我国港口总甲醇库存为120.05万吨，比同比增加了23.38万吨，但随着进口缩量的影响，该数字也在减少。目前，内地代表企业的甲醇库存也不高，且新增宝丰烯烃装置仍有外采需求，内地市场价格重心走高，对港口价格同样有一定支撑。

周禹通认为，短期看来，甲醇进口量将下降，但长期看来，随着非伊装置检修回归，限气季节过去，进口量反而存在回升的可能。从估值角度来看，煤价冬季存在一定支撑，国际甲醇供应量下降，甲醇价格重心或将上移。

同时，周禹通警告称，在供应过剩和需求疲软等因素影响下，甲醇价格可能存在回调压力，投资者需要注意这种风险。他建议投资者可以观察后期库存情况，如果后期甲醇价格仍处于高位，下游MTO利润较低但库存去化缓慢，则价格可能存在回调压力。

对此背景下，彭杰斌认为，短期看来，甲醇进口量将下降，但长期看来，随着非伊装置检修回归和限气季节过去，进口量反而存在回升的可能。他建议投资者需要注意供应过剩的风险和需求疲软的风险，同时需要观察港口库存情况，如果后市去库力度不及预期，可能增加甲醇价格下行的压力。


In [36]:
client(content= f'生成文章概要：{s}',model='marco-o1')

<Thought>
Alright, I need to generate an article summary based on the provided text. Let's break down the key points step by step to ensure a comprehensive overview.

First, the main topic is about the decline in China's methanol imports. Specifically, there's been a noticeable decrease in import volumes starting from October and November of this year. The data shows that in October and early November, monthly imports were around 130 million tons, but by the end of November, imports had significantly decreased to approximately 70 million tons for Iran and an estimated 105 million tons for the entire country in December. This is a clear decline compared to the previous two months.

The primary reason cited for this decrease is maintenance shutdowns in Iranian methanol plants. Analysts like Zhou Yutong from Yongan Futures point out that Iranian facilities are undergoing regular maintenance, leading to lower operating rates. Additionally, Iran has been facing issues with gas supply and el